# Custom Evaluation Criteria

In this example, we will see how to create a custom metric.

We will create a metric that evaluates whether a user query is decomposed into sub-queries, covering all the angles of the original query to retrieve all the necessary information.

# Creating the metric

`flow-judge` makes it easy to create custom metrics.

In [1]:
from flow_judge.metrics import CustomMetric, RubricItem
from IPython.display import Markdown, display

# Define the criteria
evaluation_criteria = """Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?"""

# Define the rubric using RubricItem's
rubric = [
    RubricItem(
        score=1,
        description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."),
    RubricItem(
        score=2,
        description="The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the main topic."),
    RubricItem(
        score=3,
        description="The sub-queries demonstrate excellent breadth, effectively covering all major aspects of the main query. They break down the main question into a diverse set of dimensions, ensuring a comprehensive exploration of the topic. Each significant facet of the main query is represented in the sub-queries, allowing for a thorough and well-rounded investigation of the subject matter."),
]

# We need to define the required inputs and output for the metric
required_inputs = ["query"]
required_output = "sub_queries"

# Create the metric
sub_query_coverage = CustomMetric(
    name="sub-query-coverage",
    criteria=evaluation_criteria,
    rubric=rubric,
    required_inputs=required_inputs,
    required_output=required_output
)

As you can see, we quickly created a custom metric that will instruct the model to evaluate according to the criteria and rubric we set.

In [2]:
sub_query_coverage

CustomMetric(name='sub-query-coverage', criteria='Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?', rubric=[RubricItem(score=1, description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."), RubricItem(score=2, description='The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the main topic.'), RubricItem(score=3, description='The sub-queries demonstrate exce

# Running evaluations with the custom metric

Now, we just need to initialize the judge with our custom metric and run the evaluations.

In [3]:
from flow_judge.flow_judge import EvalInput, FlowJudge
from flow_judge.models import Vllm #, Llamafile, Hf
# from flow_judge import Baseten

# If you are running on an Ampere GPU or newer, create a model using VLLM
model = Vllm()

# Or if not running on Ampere GPU or newer, create a model using no flash attn and Hugging Face Transformers
# model = Hf(flash_attn=False)

# Or create a model using Llamafile if not running an Nvidia GPU & running a Silicon MacOS for example
# model = Llamafile()

# Or create a model using Baseten if you don't want to run locally.
# As a pre-requisite step:
#  - Sign up to Baseten
#  - Generate an api key https://app.baseten.co/settings/api_keys
#  - Set the api key as an environment variable & initialize:
# import os
# os.environ["BASETEN_API_KEY"] = "your_api_key"
# model = Baseten()

# Initialize the judge
judge = FlowJudge(metric=sub_query_coverage, model=model)

INFO 10-08 08:52:43 awq_marlin.py:90] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
WARNING 10-08 08:52:43 config.py:389] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-08 08:52:43 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='flowaicom/Flow-Judge-v0.1-AWQ', speculative_config=None, tokenizer='flowaicom/Flow-Judge-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(gu

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 10-08 08:52:46 model_runner.py:1025] Loading model weights took 2.1717 GB
INFO 10-08 08:52:48 gpu_executor.py:122] # GPU blocks: 3085, # CPU blocks: 682


In [4]:
# Sample to evaluate

query = "I placed an order for a custom-built gaming PC (order #AC-789012) two weeks ago, but the estimated delivery date has changed twice since then. Originally it was supposed to arrive yesterday, then it got pushed to next week, and now the tracking page shows 'Status: Processing' with no delivery estimate at all. I've tried calling customer service, but after waiting on hold for an hour, I was told to check the website. Can you please look into this and explain what's causing the delays, when I can realistically expect my order to arrive, and whether I'm eligible for any kind of compensation or expedited shipping given these repeated delays? I'm especially concerned because I need this computer for an upcoming gaming tournament I'm participating in next month."
sub_queries = "What is the current shipping status of my order? How can I build a PC?"

# bad decomposition
eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = judge.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.70s/it, est. speed input: 147.77 toks/s, output: 45.67 toks/s]


In [5]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries fail to provide sufficient breadth to cover all aspects of the main query. The main query involves multiple issues: repeated delays in delivery, lack of delivery estimate, customer service experience, and potential compensation. However, the sub-queries only address two very narrow aspects: the current shipping status and how to build a PC.

The first sub-query, "What is the current shipping status of my order?" is relevant but limited. It only addresses the current status of the order, not the repeated delays or the lack of delivery estimate.

The second sub-query, "How can I build a PC?" is entirely unrelated to the main query. It diverges significantly from the main query's intent and fails to address any of the customer's concerns about the delayed order.

Using these sub-queries alone would result in a severely limited exploration of the topic. They do not cover the repeated delays, lack of delivery estimate, customer service experience, potential compensation, or the urgency due to the upcoming gaming tournament. There are noticeable gaps in coverage, with several important facets of the main query being underrepresented or missing entirely.

Therefore, the sub-queries do not meet the criteria for sufficient breadth as outlined in the scoring rubric.

__Score:__
1

In [6]:
# Good decomposition

sub_queries = """1. What is the current status of order #AC-789012, and why has it changed from the original estimated delivery date?
2. What specific factors are causing the delays in processing and shipping this custom-built gaming PC?
3. Based on the current situation, when can the customer realistically expect the order to arrive?
4. Given the repeated delays and changes in estimated delivery, what compensation options (if any) are available to the customer?
5. Is expedited shipping an option at this point, and if so, how would it affect the delivery timeline?
6. How can the urgency of this order be communicated and prioritized, considering the customer's upcoming gaming tournament next month?
7. What steps has customer service already taken to address this issue, and what additional actions can be taken to resolve it?
8. How can the customer receive more frequent and accurate updates about their order status going forward?"""

eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = judge.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.59s/it, est. speed input: 179.46 toks/s, output: 44.64 toks/s]


In [7]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries demonstrate excellent breadth, effectively covering all major aspects of the main query. Each significant facet of the main query is represented in the sub-queries, ensuring a comprehensive exploration of the topic.

The sub-queries address the following key points from the main query:

1. Current status and reasons for delivery date changes
2. Specific factors causing delays
3. Realistic expected arrival time
4. Compensation options due to repeated delays
5. Availability of expedited shipping
6. Communication of order urgency
7. Actions taken by customer service and additional steps
8. Improved communication for future updates

These sub-queries break down the main question into diverse dimensions, ensuring a thorough investigation of the subject matter. They cover the technical aspects of the order, the customer service experience, potential solutions, and future preventative measures. This comprehensive approach allows for a well-rounded exploration of the topic, addressing all major concerns raised in the main query.

The sub-queries are well-structured and cover all important aspects without straying from the main query's intent. They provide a solid foundation for a thorough investigation and resolution of the customer's issues.

__Score:__
3

In [8]:
# Okish decomposition

sub_queries = """1. What is the current status of order #AC-7892?
Why has the delivery date changed multiple times?
When will the custom-built gaming PC actually arrive?
What are your store hours on weekends?
How can I get better customer service support?
8. Can the order be expedited?
9. What is the name of the person that is responsible for the company?
"""

eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = judge.evaluate(eval_input)
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.31s/it, est. speed input: 144.97 toks/s, output: 43.77 toks/s]


__Feedback:__
The generated sub-queries demonstrate a good attempt to cover various aspects of the main query, but they fall short of providing comprehensive breadth. 

The sub-queries address several key points from the main query:
1. The current status of the order (sub-query 1)
2. The reason for multiple delivery date changes (sub-query 2)
3. The expected arrival time of the order (sub-query 3)
4. The possibility of expediting the order (sub-query 8)

However, there are noticeable gaps in coverage:
- The sub-queries do not address the customer's eligibility for compensation, which was explicitly mentioned in the main query.
- The sub-queries do not address the specific concern about the repeated delays and their impact on the customer's upcoming gaming tournament.
- The sub-queries include some irrelevant questions (e.g., store hours on weekends, the name of the person responsible for the company) that do not directly relate to the main query's concerns.

While the sub-queries touch on several important facets of the main query, they do not fully capture all the dimensions of the customer's concerns. Answering these sub-queries would provide a partial exploration of the topic, but would not fully address all aspects of the main query.

__Score:__
2